In [ ]:
import h5py
import numpy as np
import copy

file_path = "events.dl1.h5" 
file_path = "calibpipe_test/file.h5" 
file_path = "/Users/vdk/Software/ctasoft/calibpipe/src/calibpipe/tests/data/throughput/empty_muon_table.h5"
file_path = "/Users/vdk/Software/ctasoft/calibpipe/src/calibpipe/tests/data/throughput/lst_muon_table.h5"
file_path = "/Users/vdk/Software/ctapipe_processor_test/events.dl1.h5"
with h5py.File(file_path, "r") as h5_file:
    
    # List all groups in the file
    print("Groups in the file:")
    for group in h5_file:
        print(group)
    
    # You can also recursively list all datasets
    def print_structure(name, obj):
        print(f"{name}: {obj}")
    
    print("\nFile structure:")
    h5_file.visititems(print_structure)

In [ ]:
with h5py.File(file_path, "r") as h5_file:
    # Access a dataset
    dataset = h5_file["dl1/event/telescope/muon/tel_001"]
    test_dataset = h5_file.get("dl1/event/telescope/muon/tel_001")
    trigger_dataset = h5_file["dl1/event/subarray/trigger"]
    tigger_info = h5_file["dl1/event/telescope/trigger"]
    time_dataset = trigger_dataset["time"]
    trigger_lookup = trigger_dataset[:50]
    simulation_info = h5_file['configuration/simulation/run']
    simulation_dataset = simulation_info[()]
    
    simulation_images_tel = h5_file['simulation/event/telescope/images/tel_001']
    data_images_tel = h5_file['dl1/event/telescope/images/tel_001']
    print("simulation_images_tel: ", simulation_images_tel.shape)
    print("simulation_images_tel: " , simulation_images_tel.dtype)
    
    simulation_event_subarray_shower = h5_file['simulation/event/subarray/shower']
    print("simulation_event_subarray_shower: ", simulation_event_subarray_shower.shape)
    print("simulation_event_subarray_shower: " , simulation_event_subarray_shower.dtype)
    
    # Read the first and last values
    first_time = time_dataset[0]
    last_time = time_dataset[-1]
    # Check the shape and data type of the dataset
    print("Dataset shape:", dataset.shape)
    print("Dataset dtype:", dataset.dtype)
    dataset_dtype = dataset.dtype
    trigger_dataset_dtype = trigger_dataset.dtype
    trigger_info_dtype = tigger_info.dtype
    simulation_dtype = simulation_info.dtype
    # Load the data from the dataset
    data = dataset[:50]
    data_trigger = trigger_dataset[:50]
    data_trigger_info = tigger_info[:50]
    data_images_tel = data_images_tel[:50]

In [ ]:
if simulation_dtype.names:
    # If the dtype is structured, access fields by name
    first_field = simulation_dataset.dtype.names[0]
    print(f"First field ({first_field}):", data[first_field])

In [ ]:
for dtype_name in simulation_dtype.names:
    print(f"Field {dtype_name}: {simulation_dataset[dtype_name]}")

In [ ]:
('obs_id', '<i4'), ('event_id', '<i8'), ('tel_id', '<i2'), ('muonring_center_fov_lon', '<f8'), ('muonring_center_fov_lat', '<f8'), ('muonring_radius', '<f8'), ('muonring_center_phi', '<f8'), ('muonring_center_distance', '<f8'), ('muonparameters_containment', '<f8'), ('muonparameters_completeness', '<f8'), ('muonparameters_intensity_ratio', '<f4'), ('muonparameters_mean_squared_error', '<f8'), ('muonefficiency_width', '<f8'), ('muonefficiency_impact', '<f8'), ('muonefficiency_impact_x', '<f8'), ('muonefficiency_impact_y', '<f8'), ('muonefficiency_optical_efficiency', '<f8'), ('muonefficiency_is_valid', 'u1'), ('muonefficiency_parameters_at_limit', 'u1'), ('muonefficiency_likelihood_value', '<f8')

In [ ]:
data_array = np.array(data, dtype=dataset_dtype)

In [ ]:
data_list = np.array(data.tolist())
type(data_list)

In [ ]:
data_array[0]

In [ ]:
data_list[0,3:]

In [ ]:
type(data)

In [ ]:
type(data[0])

In [ ]:
from astropy.table import Table

In [ ]:
table_test = Table(data_array)

In [ ]:
table_test.columns

In [ ]:
table_test['event_id','muonring_center_fov_lon','muonring_center_fov_lat', 'muonring_radius', 'muonparameters_ring_size', 'muonparameters_size_outside','muonefficiency_optical_efficiency'][:10]

In [ ]:
from astropy.table import Table, vstack

def read_and_process_fits(files):
    dat = None
    for muon_file in files:
        dat2 = Table.read(muon_file, format='fits')
        
        # Convert columns to boolean if they exist
        for col in ['good_ring', 'is_valid', 'parameters_at_limit']:
            if col in dat2.colnames:
                dat2[col] = dat2[col].astype(bool)
        
        # Stack the tables
        dat = vstack([dat, dat2]) if dat is not None else dat2
    
    return dat

muon_table = read_and_process_fits(["/Users/vdk/Software/ctapipe_processor_test/lstchain_comparison/muon_table.fits"])

muon_table['event_id', 'ring_center_x', 'ring_center_y', 'ring_radius', 'ring_size', 'size_outside', 'muon_efficiency'][:10]

In [ ]:
muon_table.columns

In [ ]:
table_test.columns

In [ ]:
np.nanmean(table_test['muonefficiency_optical_efficiency'])

In [ ]:
np.nanmean(muon_table['muon_efficiency'])

In [ ]:
np.nanmean(table_test['muonring_center_fov_lon'])

In [ ]:
len(table_test)

In [ ]:
filtered_table = table_test[(table_test['muonefficiency_is_valid'] != 0) & (table_test['muonefficiency_parameters_at_limit'] != 1)]
filtered_table['muonefficiency_optical_efficiency']

In [ ]:
all_attributes = table_test.__dir__()
user_defined = [attr for attr in all_attributes if not attr.startswith('_')]
user_defined

In [ ]:
data_trigger[0]

In [ ]:
data_trigger['time'][0] - data_trigger['time'][-1]

In [ ]:
table_test

In [ ]:
muon_table

In [ ]:

muon_table = table_test

min_ring_radius = 0.1
max_ring_radius = 1.5
min_impact_parameter = 0.1
max_impact_parameter = 1.0
ring_completeness_threshold = 0.1
ring_containment_threshold = 0.1

mask = (
    (muon_table['muonring_radius'] >= min_ring_radius) &
    (muon_table['muonring_radius'] <= max_ring_radius) &
    (muon_table['muonefficiency_impact'] >= min_impact_parameter) &
    (muon_table['muonefficiency_impact'] <= max_impact_parameter) &
    (muon_table['muonparameters_completeness'] >= ring_completeness_threshold) &
    (muon_table['muonparameters_containment'] >= ring_containment_threshold) &
    (muon_table['muonefficiency_is_valid'] != 0) &
    (muon_table['muonefficiency_parameters_at_limit'] != 1)
) & ~(
    np.isnan(muon_table['muonring_radius']) |
    np.isnan(muon_table['muonefficiency_impact']) |
    np.isnan(muon_table['muonparameters_completeness']) |
    np.isnan(muon_table['muonparameters_containment']) |
    np.isnan(muon_table['muonefficiency_is_valid']) |
    np.isnan(muon_table['muonefficiency_parameters_at_limit'])
)

filtered_table = muon_table[mask]
filtered_table

In [ ]:
muon_table

In [ ]:
test_dict = {'1' : 0}

list(test_dict.keys())[0]